In [1]:
import pandas as pd
import numpy as np

## Mean-Variance Optimization

### Q1 Summary Statistics

In [2]:
# load data and calculate excess return

df = pd.read_excel('assetclass_data_monthly_2009.xlsX').set_index('Dates')
risky_assets = df.columns[:-1]
df_ex_ret = df[risky_assets].sub(df['Cash'],axis=0)
df_ex_ret.head()

,Domestic Equity,Foreign Equity,Emerging Markets,Private Equity,Absolute Return,High Yield,Commodities,Real Estate,Domestic Bonds,Foreign Bonds,Inflation-Indexed
Dates,,,,,,,,,,,
2009-03-31,0.082349,0.082774,0.167482,0.152342,-0.012250,0.018166,0.043914,0.034089,0.031970,0.046377,0.057925
2009-04-30,0.098793,0.114638,0.155007,0.229647,0.022329,0.137877,-0.009741,0.295595,-0.028010,0.008439,-0.018521
2009-05-29,0.058921,0.132384,0.159879,0.054357,0.028333,0.028963,0.197137,0.023195,-0.020293,0.054136,0.020491
2009-06-30,-0.001272,-0.014908,-0.023120,0.044847,-0.004035,0.032761,0.005095,-0.025683,-0.006119,0.004552,0.001410
2009-07-31,0.074632,0.100439,0.110174,0.143274,0.015351,0.069189,0.004465,0.105825,0.008340,0.031310,0.000910


In [3]:
# calculate annualized mean, vol and sharpe ratio

table1 = pd.DataFrame(columns=['mean','vol','sharpe ratio'], index=df_ex_ret.columns)
table1['mean'] = df_ex_ret.mean() * 12
table1['vol'] = df_ex_ret.std()*np.sqrt(12)
table1['sharpe ratio'] = table1['mean']/table1['vol']
table1

,mean,vol,sharpe ratio
Domestic Equity,0.156341,0.129607,1.206268
Foreign Equity,0.097513,0.157670,0.618463
Emerging Markets,0.096396,0.201714,0.477885
Private Equity,0.163689,0.199588,0.820138
Absolute Return,0.023228,0.044232,0.525135
High Yield,0.088232,0.082886,1.064493
Commodities,-0.020108,0.190936,-0.105312
Real Estate,0.174774,0.174976,0.998844
Domestic Bonds,0.037121,0.058129,0.638601
Foreign Bonds,0.025314,0.076886,0.329236


### Q2 The MV Frontier

#### (a)

In [4]:
# define a function to calcute MV portfolio weights
# diag: default parameter. if set true, use diagonal covariance matrix for MV calculation

def compute_mv(data, diag=False):
    mu_tilde = data.mean()
    covariance = data.cov()
    if diag:
        covariance = np.diag(np.diag(covariance))
    N = covariance.shape[0]
    cov_inv = np.linalg.inv(covariance)
    omega_tan = cov_inv @ mu_tilde /(np.ones(N) @ cov_inv @ mu_tilde)
    return pd.Series(omega_tan,index=data.columns)

omega_tan = compute_mv(df_ex_ret)
omega_tan

Domestic Equity      1.100132
Foreign Equity      -0.045800
Emerging Markets    -0.144565
Private Equity      -0.166304
Absolute Return     -1.166062
High Yield           0.791084
Commodities         -0.117513
Real Estate         -0.215180
Domestic Bonds       0.799114
Foreign Bonds       -0.022817
Inflation-Indexed    0.187910
dtype: float64

#### (b)

In [5]:
# define a function to calculate portfolio statistics. Default to report annual result

def portfolio_stats(omega, mu_tilde, Sigma, annualize_factor=12):
    mean = annualize_factor * mu_tilde @ omega
    vol = np.sqrt(annualize_factor) * np.sqrt(omega @ Sigma @ omega)
    sharpe_ratio = mean/vol
    df_stats = pd.DataFrame([omega.sum(), mean, vol, sharpe_ratio],
        index=['Fraction in Risky Asset', 'Mean', 'Volatlity', 'Sharpe Ratio'],
        columns=['Stat'])
    return df_stats

portfolio_stats(omega_tan, df_ex_ret.mean(), df_ex_ret.cov())

,Stat
Fraction in Risky Asset,1.000000
Mean,0.169664
Volatlity,0.071755
Sharpe Ratio,2.364485


### Q3 The Allocation

#### (a)

In [6]:
# define a function to calculate MV portfolio weights with target return. Default target return is 0.0067

def target_mv_portfolio(data,mu_p=0.0067,diag=False):
    mu_tilde = data.mean()
    covariance = data.cov()
    if diag:
        covariance = np.diag(np.diag(covariance))
    N = covariance.shape[0]
    cov_inv = np.linalg.inv(covariance)
    omega_tan = cov_inv @ mu_tilde /(np.ones(N) @ cov_inv @ mu_tilde)
    mu_tan = omega_tan @ data.mean()
    delta_tilde =mu_p/mu_tan
    omega_star = delta_tilde * omega_tan
    
    return pd.Series(omega_star,index=data.columns)

In [7]:
omega_star = target_mv_portfolio(df_ex_ret)
omega_star

Domestic Equity      0.521328
Foreign Equity      -0.021704
Emerging Markets    -0.068506
Private Equity      -0.078808
Absolute Return     -0.552571
High Yield           0.374877
Commodities         -0.055687
Real Estate         -0.101969
Domestic Bonds       0.378682
Foreign Bonds       -0.010812
Inflation-Indexed    0.089046
dtype: float64

#### (b)

In [8]:
portfolio_stats(omega_star, df_ex_ret.mean(), df_ex_ret.cov())

,Stat
Fraction in Risky Asset,0.473878
Mean,0.080400
Volatlity,0.034003
Sharpe Ratio,2.364485


#### (c)

In [9]:
omega_star.sort_values(ascending=False)

Domestic Equity      0.521328
Domestic Bonds       0.378682
High Yield           0.374877
Inflation-Indexed    0.089046
Foreign Bonds       -0.010812
Foreign Equity      -0.021704
Commodities         -0.055687
Emerging Markets    -0.068506
Private Equity      -0.078808
Real Estate         -0.101969
Absolute Return     -0.552571
dtype: float64

#### (d)  

see PDF

### Q4 Long-Short Positions

#### (a)

In [10]:
df_Q4 = df_ex_ret[['Domestic Equity','Foreign Equity']]
omega_star = target_mv_portfolio(df_Q4)
omega_star

Domestic Equity    0.769695
Foreign Equity    -0.409531
dtype: float64

In [11]:
portfolio_stats(omega_star, df_Q4.mean(), df_Q4.cov())

,Stat
Fraction in Risky Asset,0.360164
Mean,0.080400
Volatlity,0.053858
Sharpe Ratio,1.492813


#### (b)

see PDF

#### (c)

In [12]:
df_Q4_adjusted = df_Q4.copy()
df_Q4_adjusted.loc[:,'Foreign Equity'] = df_Q4.loc[:,'Foreign Equity'] + 0.001
omega_star = target_mv_portfolio(df_Q4_adjusted)   
omega_star

Domestic Equity    0.780259
Foreign Equity    -0.379738
dtype: float64

In [13]:
portfolio_stats(omega_star, df_Q4_adjusted.mean(), df_Q4_adjusted.cov())

,Stat
Fraction in Risky Asset,0.400521
Mean,0.080400
Volatlity,0.057137
Sharpe Ratio,1.407144


#### (d)

see PDF

### Q5 Robustness

In [14]:
# use diagonal covariance matrix to compute MV portfolio

omega_star = target_mv_portfolio(df_ex_ret,0.0067,True)
omega_star

Domestic Equity      0.120236
Foreign Equity       0.050674
Emerging Markets     0.030606
Private Equity       0.053085
Absolute Return      0.153376
High Yield           0.165913
Commodities         -0.007125
Real Estate          0.073746
Domestic Bonds       0.141925
Foreign Bonds        0.055320
Inflation-Indexed    0.198569
dtype: float64

In [15]:
portfolio_stats(omega_star, df_ex_ret.mean(), df_ex_ret.cov())

,Stat
Fraction in Risky Asset,1.036324
Mean,0.080400
Volatlity,0.068709
Sharpe Ratio,1.170159


#### (b) (c)

see PDF

### Q6 Out-of-Sample Performance

#### (a)

In [16]:
in_sample_df = df_ex_ret.loc[:'2016-12']
OOS_df = df_ex_ret.loc['2017-01':]

omega_in_sample = target_mv_portfolio(in_sample_df)
omega_in_sample

Domestic Equity      0.469822
Foreign Equity      -0.041204
Emerging Markets    -0.078779
Private Equity      -0.062163
Absolute Return     -0.340700
High Yield           0.318027
Commodities         -0.037083
Real Estate         -0.084349
Domestic Bonds       0.297214
Foreign Bonds       -0.062232
Inflation-Indexed    0.147943
dtype: float64

#### (b)

In [17]:
df_stats_in_sample = portfolio_stats(omega_in_sample,in_sample_df.mean(),in_sample_df.cov())
df_stats_in_sample

,Stat
Fraction in Risky Asset,0.526496
Mean,0.080400
Volatlity,0.030861
Sharpe Ratio,2.605229


#### (c)

In [18]:
df_stats_out_of_sample = portfolio_stats(omega_in_sample,OOS_df.mean(), OOS_df.cov())
df_stats_out_of_sample

,Stat
Fraction in Risky Asset,0.526496
Mean,0.050292
Volatlity,0.031903
Sharpe Ratio,1.576403


#### (d)

Discussion see PDF

In [19]:
# in-sample MV portfolio using data 2017-2019

in_sample_df_17_19 = df_ex_ret.loc['2017-01':]
omega_in_sample_17_19 = target_mv_portfolio(in_sample_df_17_19)
omega_in_sample_17_19

Domestic Equity      0.773167
Foreign Equity       0.049974
Emerging Markets     0.108954
Private Equity       0.075912
Absolute Return     -2.572672
High Yield          -0.402142
Commodities          0.110183
Real Estate          0.000042
Domestic Bonds       1.303917
Foreign Bonds        0.117154
Inflation-Indexed   -1.104505
dtype: float64

In [20]:
df_stats_in_sample_17_19 = portfolio_stats(omega_in_sample_17_19,in_sample_df_17_19.mean(),in_sample_df_17_19.cov())
df_stats_in_sample_17_19

,Stat
Fraction in Risky Asset,-1.540017
Mean,0.080400
Volatlity,0.028185
Sharpe Ratio,2.852554


In [21]:
# summary of in-sample and out-of-sample performance

(pd.DataFrame([
    df_stats_in_sample['Stat'],
    df_stats_out_of_sample['Stat'],
    df_stats_in_sample_17_19['Stat']
    ], index=['In-Sample: 2000-2016', 'Out-of-Sample', 'In-Sample: 2017-2019'])
    ).drop(columns=['Fraction in Risky Asset'])

,Mean,Volatlity,Sharpe Ratio
In-Sample: 2000-2016,0.080400,0.030861,2.605229
Out-of-Sample,0.050292,0.031903,1.576403
In-Sample: 2017-2019,0.080400,0.028185,2.852554


### Q7 Robust Out-of-Sample Performance

In [22]:
# in-sample MV portfolio using data through 2016 and diagonal covariance matrix

in_sample_df = df_ex_ret.loc[:'2016-12']
OOS_df = df_ex_ret.loc['2017-01':]
omega_in_sample = target_mv_portfolio(in_sample_df,0.0067,True)
omega_in_sample

Domestic Equity      0.113155
Foreign Equity       0.043135
Emerging Markets     0.025865
Private Equity       0.047495
Absolute Return      0.139586
High Yield           0.142795
Commodities         -0.006866
Real Estate          0.063752
Domestic Bonds       0.116634
Foreign Bonds        0.038396
Inflation-Indexed    0.172149
dtype: float64

In [23]:
df_stats_in_sample = portfolio_stats(omega_in_sample,in_sample_df.mean(),in_sample_df.cov())
df_stats_in_sample

,Stat
Fraction in Risky Asset,0.896095
Mean,0.080400
Volatlity,0.064872
Sharpe Ratio,1.239368


In [24]:
# Out-of-Sample performance 2017-2019

df_stats_out_of_sample = portfolio_stats(omega_in_sample,OOS_df.mean(),OOS_df.cov())
df_stats_out_of_sample

,Stat
Fraction in Risky Asset,0.896095
Mean,0.043298
Volatlity,0.044173
Sharpe Ratio,0.980193


In [25]:
# in-sample MV portfolio using data 2017-2019

in_sample_df_17_19 = df_ex_ret.loc['2017-01':]
omega_in_sample_17_19 = target_mv_portfolio(in_sample_df_17_19,0.0067,True)
omega_in_sample_17_19

Domestic Equity      0.136006
Foreign Equity       0.089338
Emerging Markets     0.054158
Private Equity       0.072229
Absolute Return      0.188534
High Yield           0.344375
Commodities         -0.005606
Real Estate          0.127464
Domestic Bonds       0.272808
Foreign Bonds        0.192482
Inflation-Indexed    0.342984
dtype: float64

In [26]:
df_stats_in_sample_17_19 = portfolio_stats(omega_in_sample_17_19,in_sample_df_17_19.mean(),in_sample_df_17_19.cov())
df_stats_in_sample_17_19

,Stat
Fraction in Risky Asset,1.814772
Mean,0.080400
Volatlity,0.079209
Sharpe Ratio,1.015037


In [27]:
# summary of in-sample and out-of-sample performance using diagonal matrix

(pd.DataFrame([
    df_stats_in_sample['Stat'],
    df_stats_out_of_sample['Stat'],
    df_stats_in_sample_17_19['Stat']
    ], index=['In-Sample: 2000-2016', 'Out-of-Sample', 'In-Sample: 2017-2019'])
    ).drop(columns=['Fraction in Risky Asset'])

,Mean,Volatlity,Sharpe Ratio
In-Sample: 2000-2016,0.080400,0.064872,1.239368
Out-of-Sample,0.043298,0.044173,0.980193
In-Sample: 2017-2019,0.080400,0.079209,1.015037
